In [1]:

import pandas as pd
import pymongo
from decouple import config

In [6]:
share_events_df = pd.read_csv("orig_shared_events.csv")
share_events_df.head(20)

,Unnamed: 0,_id,server_upload_time,server_received_time,event_type,user_creation_time,city,language,device_model,processed_time,...,event_properties_categories,user_properties_organization___v,user_properties_organization_status,user_properties_organization_logo_url_url,user_properties_organization_owner_id,user_properties_organization_updated_at,user_properties_organization_code,user_properties_organization_created_at,domain,insert_key
0,0,6101a7cf67189533badad509,1620034324392,1620034324384,Share Presentation:Share Center,1617104362001,Tananger,Norwegian,Mac,1620034326331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stream.masterwizr.com,NaN
1,1,6101a7cf67189533badad512,1620204999636,1620204999630,Share Template Show Room:Studio,1620204999761,Stavanger,English,Windows,1620205001489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,studio.masterwizr.com,NaN
2,2,6101a7cf67189533badad52f,1624542558735,1624542558728,Share Presentation:Content Admin,1615276990031,Tirana,English,Windows,1624542561706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stream.masterwizr.com,NaN
3,3,6101a7cf67189533badad4fd,1619102304194,1619102304183,Share Template Show Room:Studio,1619102164202,Ytre Arna,Norwegian,Windows,1619102306093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,studio.masterwizr.com,NaN
4,4,6101a7cf67189533badad505,1619531913152,1619531913138,Share Presentation:Content Admin,1616399457149,Oslo,Norwegian,Mac,1619531916312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stream.masterwizr.com,NaN


In [11]:
countries_df = share_events_df.groupby("country").agg("sum")
countries_df

,Unnamed: 0,server_upload_time,server_received_time,user_creation_time,processed_time,session_id,event_time,schema,client_event_time,os_version,...,event_id,global_user_properties,user_properties_session_id,user_properties_organization___v,user_properties_organization_status,user_properties_organization_logo_url_url,user_properties_organization_owner_id,user_properties_organization_updated_at,user_properties_organization_code,user_properties_organization_created_at
country,,,,,,,,,,,,,,,,,,,,,
Albania,3307,45489005858010,45489005857703,45357584096602,45489005917380,45488960194962,45489005745834,336,45489005709042,2544,...,238,0.0,4.548896e+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bangladesh,117147,384263070096654,384263070093756,384195013713690,384263070459159,384262493184938,384263068035265,2832,384263063974027,21712,...,9673,0.0,3.842625e+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Benin,708,1630152336815,1630152336789,1630055725378,1630152338938,1630152060258,1630152336351,12,1630152336351,92,...,5,0.0,1.630152e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kenya,2475,8139901464581,8139901464540,8139901293400,8139901477366,8139900474485,8139901463605,60,8139901463605,460,...,15,0.0,8.139900e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kosovo,13901,34217957370725,34217957370464,34106439648030,34217957402246,34217933965495,34217957169517,252,34217957104167,1932,...,680,0.0,3.421793e+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Netherlands,176,8123144865393,8123144865313,8123144742645,8123144870980,8123144742645,8123144854858,60,8123144854858,455,...,15,0.0,8.123131e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Norway,115014,642794127760922,642794127756521,639973005692192,642794128573601,642793260662473,642794122432503,4740,642794122432503,35793,...,61426,0.0,6.427923e+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Serbia,2818,6520214238539,6520214238463,6485681851108,6520214244617,6520213474948,6520214237020,48,6520214237020,368,...,154,0.0,6.520213e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Uganda,3294,40581074008626,40581074008288,40535691373594,40581074054186,40581063393382,40581073990076,300,40581073990076,2256,...,98,0.0,4.058106e+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
countries = share_events_df["country"].unique()
countries

array(['Norway', 'Albania', 'Netherlands', 'Uganda', 'Bangladesh',
       'Kenya', 'Kosovo', 'Serbia', 'Benin'], dtype=object)

In [17]:
# Extracting `user_properties` data
users_df = share_events_df.loc[
    :, share_events_df.columns.str.startswith("user_properties")
]
users_df["user_properties_user_id"] = share_events_df[
    "user_id"
]  # adding user_id to events data because we will need it as a foreign key to the ShareEvents table

# Prepare columns from `user_properties`  that will later be dropped from the initial dataset
user_cols_to_drop = list(users_df.columns)

# Rename `user_properties` columns to remove prefix
prefix = "user_properties_"
users_df.rename(columns=lambda x: x[len(prefix) :] if x.startswith(prefix) else x, inplace=True)

# Prepare to create user table
users_df.reset_index(drop=True, inplace=True)
users_df.drop_duplicates(subset=["user_id"], inplace=True)

users_df

/var/folders/32/t8k4f_4d55nds458_sr5k_d80000gn/T/ipykernel_2590/1041634214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_df["user_properties_user_id"] = share_events_df[
/opt/homebrew/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/opt/homebrew/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,email,name,user_unique_identifier,device_id,session_id,organization_name,organization_type,organization__id,role_id,organization___v,organization_status,organization_logo_url_url,organization_owner_id,organization_updated_at,organization_code,organization_created_at,user_id
0,leif@masterwizr.no,Leif Birger Kleppa,943e2fc4-a66b-42c8-a7c6-81380a0a50b1,-Pco2mlaPZGQDdOEgKi9IY,1.620034e+12,mw-intranet,Company,5eff44d625053a0dc1c7bb98,5db8291717a3e1072298a291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5f6082b63454bc01ebd20318
1,NaN,NaN,69dc57bf-5e52-4a07-a811-480e26e2c7d1,rbDpjPpB46N4_rBi2Oq5Sf,1.620205e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69dc57bf-5e52-4a07-a811-480e26e2c7d1
2,info@masterwizr.com,Jone Smedsvig,a9866417-93b6-484d-9619-714bb34b4b4a,UIMMG1fEsYBH3bO6PLEDtq,1.624541e+12,mw-sales,Company,5f8670241916d600128dcfc0,5db8291717a3e1072298a291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5eff451125053a0dc1c7bc52
3,NaN,NaN,eb41d33c-4552-47c2-b931-9ebdd74b4297,dZwxHg0a1s8FV050nTrmVT,1.619100e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eb41d33c-4552-47c2-b931-9ebdd74b4297
4,nico@masterwizr.no,Nico Rasmussen,77640c84-db18-4f33-9b24-a201dea87991,RmU7tllO5CIQQZkxfyhGPp,1.619532e+12,mw-sales,Company,5f8670241916d600128dcfc0,5db8291717a3e1072298a291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5f5a16c659be2422870c53cc
6,NaN,NaN,19188517-81c8-4301-ad48-2f3b98b7554b,UIMMG1fEsYBH3bO6PLEDtq,1.621420e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19188517-81c8-4301-ad48-2f3b98b7554b
7,NaN,NaN,61e73a72-ad58-4886-8068-683386eb4845,vh-vWno25kzvSZ7ccWHUh9,1.624255e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61e73a72-ad58-4886-8068-683386eb4845
9,NaN,NaN,a4f1d288-7b54-44b2-a8ab-008a15ee6880,MIS5QYjCHYBBLTYFkvjaWM,1.624626e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a4f1d288-7b54-44b2-a8ab-008a15ee6880
11,NaN,NaN,043ab0b9-c8c6-4875-a2a7-8e5e4548f434,LHMpapwO-dKDFSjcBzyt5o,1.621923e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,043ab0b9-c8c6-4875-a2a7-8e5e4548f434
15,NaN,NaN,93dcf069-952a-4504-995d-a32e810ec14b,9R-ZZyuIAYA8Tk5HW3pVfJ,1.619698e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93dcf069-952a-4504-995d-a32e810ec14b


In [21]:
organizations = users_df[["organization__id" ,"organization_name"]]
organizations 

,organization__id,organization_name
0,5eff44d625053a0dc1c7bb98,mw-intranet
1,NaN,NaN
2,5f8670241916d600128dcfc0,mw-sales
3,NaN,NaN
4,5f8670241916d600128dcfc0,mw-sales
6,NaN,NaN
7,NaN,NaN
9,NaN,NaN
11,NaN,NaN
15,NaN,NaN


In [23]:
share_events_df[["user_properties_organization__id" ,"user_properties_organization_name"]]

,user_properties_organization__id,user_properties_organization_name
0,5eff44d625053a0dc1c7bb98,mw-intranet
1,NaN,NaN
2,5f8670241916d600128dcfc0,mw-sales
3,NaN,NaN
4,5f8670241916d600128dcfc0,mw-sales
...,...,...
715,NaN,NaN
716,NaN,NaN
717,NaN,NaN
718,NaN,NaN
